In [1]:
from django.contrib.staticfiles.templatetags.staticfiles import static
invoice = Invoice.objects.first()
member = SubscriptionMember.objects.first()
user= User.objects.first()

In [2]:
from accounting.email import *

In [3]:
email_receipt(member, invoice)

'/static_files/invoice.tpl'